# Load Erp Data

Find and format File

erpdata=pd.read_csv('../1-AD/3-DUNS/duns_denormalized.csv',sep='|',encoding='utf-8',
                    dtype={
                        'companyname':str,
                          'countrycode':str,
                          'countryname':str,
                           'street':str,
                           'city':str,
                          'postalcode':str,
                          'registerid':str,
                          'source_level':str,
                            'source_relation':str,
                          'isglobalroot':bool})

In [3]:
filepath='../1-AD/6-sourcingreport/'
filename='2017-10-18 ERP supplier Linkurious_2POR.xlsx'
erpdata=pd.read_excel(filepath+filename)

In [6]:
erpdata.dropna(subset=['companysystemid'],inplace=True)
erpdata.fillna('notavailable',inplace=True)

In [7]:
import neatmartinet as nm
for c in ['dunsid']:
    erpdata[c]=erpdata[c].apply(lambda r:nm.convert_int_to_str(n=r,fillwithzeroes=9))

In [8]:
neoimportpath='../../Neo4j/sourcingreport.graphdb/import/'
erpdata.to_csv(neoimportpath+'erpdata_commasep.csv',encoding='utf-8',sep=',',index=False)

In [14]:
erpdata['postalcode']='notavailable'
erpdata['street']='notavailable'
erpdata['registerid']='notavailable'

In [15]:
erpdata.isnull().sum()

companysystemid    0
systemid           0
companylocalid     0
companyname        0
city               0
countryname        0
countrycode        0
dunsid             0
turnover           0
ordervolume        0
postalcode         0
street             0
registerid         0
dtype: int64

# Load in neo4j

### Connect to graph

In [17]:
import py2neo
graph = py2neo.Graph(user='neo4j',password='admin')

#### Load Erp

In [19]:
query_uploaderp='''
USING PERIODIC COMMIT 10000
  LOAD CSV WITH HEADERS FROM 'file:///erpdata_commasep.csv' AS row
    MERGE (c:ErpID{companysystemid: row.companysystemid})
    ON CREATE SET
        c.companysystemid=row.companysystemid,
        c.systemid=row.systemid,
        c.companylocalid=row.companylocalid,
        c.dunsid=row.dunsid,
        c.companyname=row.companyname,
        c.countrycode=row.countrycode,
        c.countryname=row.countryname,
        c.street=row.street,
        c.city=row.city,
        c.postalcode=row.postalcode,
        c.registerid=row.registerid,
        c.turnover=toFloat(row.turnover),
        c.ordervolume=toFloat(row.ordervolume),
        c.creationdate=timestamp(),
        c.lastseen=timestamp()
    ON MATCH SET
        c.lastseen=timestamp(),
        c.spent=toFloat(row.spent);
'''
a=graph.run(query_uploaderp)
print(a.stats())

{'constraints_added': 0, 'constraints_removed': 0, 'indexes_added': 0, 'indexes_removed': 0, 'labels_added': 86696, 'labels_removed': 0, 'nodes_created': 86696, 'nodes_deleted': 0, 'properties_set': 1127048, 'relationships_deleted': 0, 'relationships_created': 0, 'contains_updates': True}


#### Create Relations

In [20]:
query_erphasduns='''
MATCH (a:ErpID),(b:DunsID)
WHERE b.dunsid = a.dunsid
MERGE (a)-[:hasDuns]->(b)
'''
a=graph.run(query_erphasduns)
print(a.stats())

{'constraints_added': 0, 'constraints_removed': 0, 'indexes_added': 0, 'indexes_removed': 0, 'labels_added': 0, 'labels_removed': 0, 'nodes_created': 0, 'nodes_deleted': 0, 'properties_set': 0, 'relationships_deleted': 0, 'relationships_created': 72457, 'contains_updates': True}


# TO DO : 
- Create Country relations
- Write relation for RegisterId
- Deal with NDM999999